# Campeonato Brasileiro - Série A

## Radom Forest Classifier

### 1. Import packages

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
from sklearn.ensemble import RandomForestClassifier

In [102]:
from sklearn.ensemble import AdaBoostClassifier

In [3]:
from sklearn.model_selection import cross_val_score

In [4]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score, recall_score
from sklearn.metrics import f1_score

In [5]:
from sklearn import datasets

### 2. Reading data

In [6]:
jogos = pd.read_csv("data/jogos-resultado-posicao.csv",sep=",")

In [7]:
del jogos["Unnamed: 0"]

In [8]:
#jogos.query('ano > 2012', inplace=True)

In [9]:
jogos.head(3)

,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante,resultado,posicao_m,posicao_v,aproveitamento_5r_v,aproveitamento_5r_m,aproveitamento_10r_v,aproveitamento_10r_m
0,brasileiro-a,2017,30,23-10,Botafogo,Corinthians,2,1,MANDANTE,6.0,1.0,0.40,0.47,0.40,0.63
1,brasileiro-a,2017,30,22-10,Chapecoense,Fluminense,2,0,MANDANTE,11.0,13.0,0.47,0.47,0.40,0.43
2,brasileiro-a,2017,30,22-10,Ponte Preta,Avaí,1,2,VISITANTE,18.0,16.0,0.13,0.27,0.43,0.30


In [10]:
jogos.keys()

Index(['campeonato', 'ano', 'rodada', 'data', 'mandante', 'visitante',
       'gols_mandante', 'gols_visitante', 'resultado', 'posicao_m',
       'posicao_v', 'aproveitamento_5r_v', 'aproveitamento_5r_m',
       'aproveitamento_10r_v', 'aproveitamento_10r_m'],
      dtype='object')

In [11]:
times = np.concatenate((jogos["visitante"].unique(), jogos["mandante"].unique()), axis=0)

In [12]:
times_dict = dict(zip(times, range(1, len(times))))

In [13]:
jogos = jogos.applymap(lambda s: times_dict.get(s) if s in times_dict else s)

### 3. Define target and data subset

In [14]:
jogos_X_df = jogos.copy()

In [15]:
del jogos_X_df["resultado"]

In [16]:
del jogos_X_df["campeonato"]

In [17]:
del jogos_X_df["data"]

In [18]:
del jogos_X_df["gols_mandante"]

In [19]:
del jogos_X_df["gols_visitante"]

In [20]:
del jogos_X_df["aproveitamento_5r_v"]

In [21]:
del jogos_X_df["aproveitamento_5r_m"]

In [22]:
jogos_X_df.head()

,ano,rodada,mandante,visitante,posicao_m,posicao_v,aproveitamento_10r_v,aproveitamento_10r_m
0,2017,30,42,57,6.0,1.0,0.40,0.63
1,2017,30,43,58,11.0,13.0,0.40,0.43
2,2017,30,44,56,18.0,16.0,0.43,0.30
3,2017,30,45,53,9.0,15.0,0.23,0.40
4,2017,30,46,55,12.0,17.0,0.47,0.40


In [23]:
jogos_X_df.keys()

Index(['ano', 'rodada', 'mandante', 'visitante', 'posicao_m', 'posicao_v',
       'aproveitamento_10r_v', 'aproveitamento_10r_m'],
      dtype='object')

In [24]:
y = jogos["resultado"].values

In [25]:
X = jogos_X_df.values

### 4. Train and Test Split

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y, 
                                        test_size=0.2, 
                                        random_state=1)

### 4.1 Grid Search

In [28]:
from sklearn.grid_search import GridSearchCV

/home/thiago/anaconda2/envs/py3k/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/thiago/anaconda2/envs/py3k/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [29]:
from sklearn.datasets import make_classification

In [30]:
from sklearn.ensemble import RandomForestClassifier

In [31]:
rfc = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=50, oob_score = True)

In [49]:
param_grid = {     
    'n_estimators': [800],
    'min_samples_leaf': [10],
    'max_features': ['log2'],
    'n_jobs': [-1],
    'min_samples_split': [3],
    'warm_start': [False],
    'oob_score':  [False], 
    'bootstrap': [True],
    'criterion': ['entropy'],
    'class_weight': ['balanced', 'balanced_subsample', None]
}

In [50]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)

In [51]:
CV_rfc.fit(X, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=-1, oob_score=True, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [800], 'min_samples_leaf': [10], 'max_features': ['log2'], 'n_jobs': [-1], 'min_samples_split': [3], 'warm_start': [False], 'oob_score': [False], 'bootstrap': [True], 'criterion': ['entropy'], 'class_weight': ['balanced', 'balanced_subsample', None]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [52]:
print (CV_rfc.best_params_)

{'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_features': 'log2', 'min_samples_leaf': 10, 'min_samples_split': 3, 'n_estimators': 800, 'n_jobs': -1, 'oob_score': False, 'warm_start': False}


### 5. Apply Random Forest Classifier

In [95]:
rf_model = RandomForestClassifier(random_state=1, criterion='entropy', max_features='log2', n_estimators=800, min_samples_leaf=10)

In [97]:
rf_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='log2', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=10,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=800, n_jobs=1, oob_score=False, random_state=1,
            verbose=0, warm_start=False)

### 6. Model Cross-Validation

In [98]:
cv_scores_rf = cross_val_score(rf_model, X_train, y_train, 
                               cv=5,scoring="accuracy")
cv_scores_rf

array([ 0.60227273,  0.58677686,  0.59710744,  0.57394002,  0.59274611])

In [99]:
cv_scores_rf_mean =  np.mean(cv_scores_rf)
cv_scores_rf_mean

0.59056863189310982

### Apply AdaBoostClassifier

In [171]:
abc_model = AdaBoostClassifier(random_state=1, n_estimators=50, learning_rate=1., algorithm="SAMME.R")

In [172]:
abc_model.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=1)

In [173]:
cv_scores_abc = cross_val_score(abc_model, X_train, y_train, 
                               cv=5,scoring="accuracy")
cv_scores_abc

array([ 0.60020661,  0.58264463,  0.56404959,  0.56670114,  0.59689119])

In [174]:
cv_scores_abc_mean =  np.mean(cv_scores_abc)
cv_scores_abc_mean

0.58209863113898097

### 7. Model Test

In [182]:
y_test_pred = rf_model.predict(X_test)

In [183]:
y_test.shape

(1210,)

In [184]:
# Results
pd.crosstab(y_test, y_test_pred,
            rownames=['Actual'], 
            colnames=['Predicted'])

Predicted,EMPATE,MANDANTE,VISITANTE
Actual,,,
EMPATE,26,214,72
MANDANTE,13,561,43
VISITANTE,19,143,119


In [90]:
confusion_matrix(y_test, y_test_pred)

array([[ 33, 211,  68],
       [ 12, 558,  47],
       [ 18, 144, 119]])

In [91]:
accuracy_score_test = accuracy_score(y_test, y_test_pred)
accuracy_score_test

0.58677685950413228

In [92]:
#precision_score_forest = precision_score(y_test, y_test_pred)
#precision_score_forest

In [93]:
#recall_score_forest = recall_score(y_test, y_test_pred)
#recall_score_forest

In [94]:
#f1_score_forest = f1_score(y_test, y_test_pred)
#f1_score_forest

### 8. Comparison accuracy: Validation vs Test

In [95]:
comparison = {"Validation": [cv_scores_rf_mean],
             "Test": [accuracy_score_test]}
pd.DataFrame(comparison, index = ["Accuracy"])

,Test,Validation
Accuracy,0.586777,0.584158


### 9. Feature Importance

In [96]:
rf_model.feature_importances_

array([ 0.0528757 ,  0.10256726,  0.070121  ,  0.07513879,  0.25843594,
        0.24863018,  0.08887672,  0.1033544 ])

In [97]:
# View a list of the features and their importance scores
imp_list = list(zip(jogos_X_df.keys(), 
                    rf_model.feature_importances_))
imp_df = pd.DataFrame(imp_list, columns = ["Features", "Importance"])
imp_df.sort_values(by = "Importance", ascending = False)

,Features,Importance
4,posicao_m,0.258436
5,posicao_v,0.248630
7,aproveitamento_10r_m,0.103354
1,rodada,0.102567
6,aproveitamento_10r_v,0.088877
3,visitante,0.075139
2,mandante,0.070121
0,ano,0.052876


### 10. Predict

In [175]:
X[0]

array([  2.01700000e+03,   3.00000000e+01,   4.20000000e+01,
         5.70000000e+01,   6.00000000e+00,   1.00000000e+00,
         4.00000000e-01,   6.30000000e-01])

In [179]:
jogos.head(10)

,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante,resultado,posicao_m,posicao_v,aproveitamento_5r_v,aproveitamento_5r_m,aproveitamento_10r_v,aproveitamento_10r_m
0,brasileiro-a,2017,30,23-10,42,57,2,1,MANDANTE,6.0,1.0,0.40,0.47,0.40,0.63
1,brasileiro-a,2017,30,22-10,43,58,2,0,MANDANTE,11.0,13.0,0.47,0.47,0.40,0.43
2,brasileiro-a,2017,30,22-10,44,56,1,2,VISITANTE,18.0,16.0,0.13,0.27,0.43,0.30
3,brasileiro-a,2017,30,22-10,45,53,2,1,MANDANTE,9.0,15.0,0.40,0.27,0.23,0.40
4,brasileiro-a,2017,30,22-10,46,55,2,1,MANDANTE,12.0,17.0,0.47,0.53,0.47,0.40
5,brasileiro-a,2017,30,22-10,47,59,1,3,VISITANTE,5.0,10.0,0.47,0.67,0.50,0.67
6,brasileiro-a,2017,30,22-10,48,54,1,3,VISITANTE,4.0,2.0,0.67,0.47,0.60,0.37
7,brasileiro-a,2017,30,22-10,49,60,1,0,MANDANTE,3.0,20.0,0.27,0.60,0.47,0.50
8,brasileiro-a,2017,30,22-10,50,52,2,0,MANDANTE,14.0,7.0,0.53,0.47,0.57,0.50
9,brasileiro-a,2017,30,21-10,51,61,1,1,EMPATE,8.0,19.0,0.27,0.73,0.20,0.60


In [202]:
rf_model.predict([2017, 31, 54, 47, 12, 6, 0.50, 0.50])

/home/thiago/anaconda2/envs/py3k/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array(['MANDANTE'], dtype=object)